In [1]:
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [2]:
import folium

from main import MapItems, Locator, MapMaker

In [3]:
locator = Locator()

In [26]:
mm = MapMaker(map_item_filename='iceland_map_items.json')
mm.main()

In [27]:
mm.base_map

In [25]:
locator("Videy", "iceland")

Getting location for Videy iceland...
Success!


[64.16521945, -21.85336698672562]

In [ ]:
Hallgrimskerk 		-> Kaartje kopen voor de lift, naar 70 m hoogte.
Arnarholl 		-> Heuvel met uitzicht over de (oude) haven.
Harpa 			-> Futuristisch gebouw.
Vissershaven
Bryggjan Brugghus	-> Brouwerij + evt. proeverij (https://www.bryggjanbrugghus.is/en)
Albinghusio		-> Parlementsgebouw.
Botanische tuinen	-> ‘s zomers tot 22.00 geopend.
Videy en Lundey	-> Eilanden die per boot te bezoeken zijn. Op Videy kan je uitstappen.

In [28]:
mm.save_map('IcelandMap.html')

In [38]:
mm.mi.to_json()

# Hotels

In [6]:
locator("Borealis hotel", "iceland")

Getting location for Borealis hotel iceland...
Success!


[64.1012758, -20.9964387]

# Availability table

In [5]:
hotels = mm.mi.d['hotels']

In [6]:
import re

In [7]:
def parse_date1(s):
    regex = r"(\d+)\-(\d+) (Juli|Aug)"
    result = re.findall(regex, s)
    if len(result) == 0:
        print(f"1: Could not extract dates from {s}")
        return None
    else:
        return result[0]

def parse_date2(s):
    regex = r"(\d+) (Juli) \- (\d+) (Aug)"
    result = re.findall(regex, s)
    if len(result) == 0:
        print(f"2: Could not extract dates from {s}")
        return None
    else:
        return result[0]

In [10]:
availability = dict()
for key, val in hotels.items():
    availability[key] = []
    for date in val['beschikbaarheid']:
        _date = parse_date1(date)
        if _date is None:
            _date = parse_date2(date)
        if _date is not None:
            availability[key].append(_date)

1: Could not extract dates from (min. 2 nachten)
2: Could not extract dates from (min. 2 nachten)
1: Could not extract dates from (min. 2 nachten)
2: Could not extract dates from (min. 2 nachten)
1: Could not extract dates from 31 Juli - 6 Aug
1: Could not extract dates from 25 Juli - 3 Aug
1: Could not extract dates from 27 Juli - 6 Aug
1: Could not extract dates from 31 Juli - 1 Aug
1: Could not extract dates from 30 Juli - 1 Aug


In [11]:
juli = list(range(18, 32))
aug = list(range(1, 4))
_juli = ["Juli"] * (len(juli) - 1)
_aug = ["Aug"] * (len(aug) - 1)

In [12]:
dates = list(zip(juli[:-1], _juli, juli[1:], _juli)) + [(juli[-1], _juli[0], aug[0], _aug[0])] + list(zip(aug[:-1], _aug, aug[1:], _aug))
fmt = "{} {} - {} {}"
dates = [fmt.format(*_date) for _date in dates]
# dates

In [13]:
import pandas as pd
idx = pd.IndexSlice

In [14]:
av = dict()
for key, val in availability.items():
    av[key] = []
    for date in val:
        if len(date) == 3:
            start_date, end_date, month = date
            start_month = month
            end_month = month
        else:
            start_date, start_month, end_date, end_month = date
        
        start_date = int(start_date)
        end_date = int(end_date)
        
        if start_date == end_date - 1:
            av[key].append(fmt.format(start_date, start_month, end_date, end_month))
        else:
            if start_month == end_month:
                ranges = list(range(start_date, end_date + 1))
                av[key].extend([fmt.format(*_date) for _date in list(zip(ranges[:-1], [start_month] * (len(ranges) - 1), ranges[1:], [end_month] * (len(ranges) - 1)))])
            else:
                ranges = list(range(start_date, 32))
                av[key].extend([fmt.format(*_date) for _date in list(zip(ranges[:-1], [start_month] * (len(ranges) - 1), ranges[1:], [start_month] * (len(ranges) - 1)))])
                
                av[key].append(fmt.format(31, start_month, 1, end_month))
                
                ranges = list(range(1, end_date + 1))
                av[key].extend([fmt.format(*_date) for _date in list(zip(ranges[:-1], [end_month] * (len(ranges) - 1), ranges[1:], [end_month] * (len(ranges) - 1)))])    

In [15]:
df_av = dict()
for hotel, d in av.items():
    df_av[hotel] = [_date in d for _date in dates]
df = pd.DataFrame(index=dates, data=df_av).T
df = pd.concat([df, pd.DataFrame(data={key: [val['region']] for key, val in hotels.items()}).T], axis=1).rename(columns={0: "Regio"}).set_index("Regio", append=True).swaplevel(0, 1).sort_index()
df

18 Juli - 19 Juli  19 Juli - 20 Juli  \
Regio                                                                
mid   Blaskogabyggd                       False              False   
      Fludir                              False              False   
      Laugarvatn                           True               True   
n     Akureyri                            False              False   
      Elda Guesthouse                     False              False   
      Godafoss                            False               True   
      Hvitafell                           False              False   
o     Egilsstadir Guesthouse               True               True   
      Gistihusid                          False              False   
w     Grundarfjordur                       True               True   
      Grundarfjordur hostel                True               True   
      Sudur-bar                           False              False   
z     Asolfsskali                          True               True   
      Eyjafjallajokull                     True              False   
      Skogar                               True              False   
zo    Hofn Guesthouse                     False              False   
      Hofn Hostel                         False              False   
      Saudanes                            False              False   
      Saudanes 2                          False              False   
zw    Grettir guesthouse                   True               True   
      Grindavik                            True              False   
      Keflavik                            False               True   

                              20 Juli - 21 Juli  21 Juli - 22 Juli  \
Regio                                                                
mid   Blaskogabyggd                       False              False   
      Fludir                              False              False   
      Laugarvatn                           True              False   
n     Akureyri                            False              False   
      Elda Guesthouse                     False              False   
      Godafoss                             True               True   
      Hvitafell                           False              False   
o     Egilsstadir Guesthouse              False              False   
      Gistihusid                          False              False   
w     Grundarfjordur                       True              False   
      Grundarfjordur hostel                True               True   
      Sudur-bar                           False               True   
z     Asolfsskali                          True               True   
      Eyjafjallajokull                     True               True   
      Skogar                               True               True   
zo    Hofn Guesthouse                     False              False   
      Hofn Hostel                         False              False   
      Saudanes                            False              False   
      Saudanes 2                          False              False   
zw    Grettir guesthouse                   True              False   
      Grindavik                            True               True   
      Keflavik                             True               True   

                              22 Juli - 23 Juli  23 Juli - 24 Juli  \
Regio                                                                
mid   Blaskogabyggd                       False              False   
      Fludir                              False              False   
      Laugarvatn                           True               True   
n     Akureyri                            False               True   
      Elda Guesthouse                     False              False   
      Godafoss                            False              False   
      Hvitafell                           False               True   
o     Egilsstadir Guesthouse          

In [16]:
route = ["zw", "mid", "w", "n", "o", "zo", "z"]

In [17]:
df.loc[idx[route, :]].astype(int)

18 Juli - 19 Juli  19 Juli - 20 Juli  \
Regio                                                                
zw    Grettir guesthouse                      1                  1   
      Grindavik                               1                  0   
      Keflavik                                0                  1   
mid   Blaskogabyggd                           0                  0   
      Fludir                                  0                  0   
      Laugarvatn                              1                  1   
w     Grundarfjordur                          1                  1   
      Grundarfjordur hostel                   1                  1   
      Sudur-bar                               0                  0   
n     Akureyri                                0                  0   
      Elda Guesthouse                         0                  0   
      Godafoss                                0                  1   
      Hvitafell                               0                  0   
o     Egilsstadir Guesthouse                  1                  1   
      Gistihusid                              0                  0   
zo    Hofn Guesthouse                         0                  0   
      Hofn Hostel                             0                  0   
      Saudanes                                0                  0   
      Saudanes 2                              0                  0   
z     Asolfsskali                             1                  1   
      Eyjafjallajokull                        1                  0   
      Skogar                                  1                  0   

                              20 Juli - 21 Juli  21 Juli - 22 Juli  \
Regio                                                                
zw    Grettir guesthouse                      1                  0   
      Grindavik                               1                  1   
      Keflavik                                1                  1   
mid   Blaskogabyggd                           0                  0   
      Fludir                                  0                  0   
      Laugarvatn                              1                  0   
w     Grundarfjordur                          1                  0   
      Grundarfjordur hostel                   1                  1   
      Sudur-bar                               0                  1   
n     Akureyri                                0                  0   
      Elda Guesthouse                         0                  0   
      Godafoss                                1                  1   
      Hvitafell                               0                  0   
o     Egilsstadir Guesthouse                  0                  0   
      Gistihusid                              0                  0   
zo    Hofn Guesthouse                         0                  0   
      Hofn Hostel                             0                  0   
      Saudanes                                0                  0   
      Saudanes 2                              0                  0   
z     Asolfsskali                             1                  1   
      Eyjafjallajokull                        1                  1   
      Skogar                                  1                  1   

                              22 Juli - 23 Juli  23 Juli - 24 Juli  \
Regio                                                                
zw    Grettir guesthouse                      1                  1   
      Grindavik                               0                  1   
      Keflavik                                0                  1   
mid   Blaskogabyggd                           0                  0   
      Fludir                                  0                  0   
      Laugarvatn                              1                  1   
w     Grundarfjordur                          0                  1   
      Grundarfjordur hostel           

# Route planner

In [6]:
locator('Reykjavik iceland', "")

Getting location for Reykjavik iceland ...
Success!


[64.145981, -21.9422367]

In [2]:
import openrouteservice as ors

In [7]:
key = "5b3ce3597851110001cf624893835ff4fbe946479421c162eb9ce6e7"

In [9]:
client = ors.Client(key=key)

In [21]:
home = locator("pastoriestraat 71a Eindhoven", "")

Getting location for pastoriestraat 71a Eindhoven ...
Success!


In [22]:
work = locator("Esp 405 Eindhoven", "")

Getting location for Esp 405 Eindhoven ...
Success!


In [27]:
work

[51.4879542, 5.4886899]

In [28]:
#set location coordinates in longitude,latitude order
coords = ((80.21787585263182,6.025423265401452),(80.23929481745174,6.019639381180123))
#call API
res = client.directions(coords)

In [ ]:
# result = client.directions((home[::-1], work[::-1]), profile='cycling-regular')
result = client.directions((home[::-1], work[::-1]), profile='driving-car')

In [30]:
geometry = result['routes'][0]['geometry']

In [31]:
geometry

'mj`yHspl`@?WB_@TyD?KAQ?g@AYAW?YK?eBRM?gEb@oBRgHj@gCPS@E?SD_@DC?OJgANm@Je@JeD`@}BZaAFu@Ha@DM?MBE?E@C?SDG@E?cAP_@Jq@H}BXuC`@gAJ}ALiBTG@I@OBQBQBwEn@K@gALqCZKASOGCCAC@Ag@KqO?SA]?K?W?K?W?EK@I?W?U@O?G?BkDAcGGaCCcC?IAe@ByAG}JCgEAa@AS?IB[@K@YB}D?WAMAYAQENk@LW@aABaELgZ{C{CgAwEwACEICC?c@EKAGBgAF_CNkAGoC[mC[cAWk@G_@Bo@Fk@?kAM{BWuDa@gBOqASYIk@[YWgA}Ai@aAg@i@[QUEo@DqBh@U@UIGZAJEDKD[NIDE@kAiJg@JUEaBUAR[zKCHQ?u@?qBYc@I_@vD@nAj@rD'

In [ ]:
ors.convert.decode_polyline(geometry)['coordinates']

In [35]:
import pandas as pd

In [57]:
["driving-car", "driving-hgv", "foot-walking", "foot-hiking", "cycling-regular", "cycling-road", "cycling-mountain", "cycling-electric"]

['driving-car',
 'driving-hgv',
 'foot-walking',
 'foot-hiking',
 'cycling-regular',
 'cycling-road',
 'cycling-mountain',
 'cycling-electric']

In [ ]:

# distance_txt = "<h4> <b>Distance :&nbsp" + "<strong>"+str(round(res['routes'][0]['summary']['distance']/1000,1))+" Km </strong>" +"</h4></b>"
# duration_txt = "<h4> <b>Duration :&nbsp" + "<strong>"+str(round(res['routes'][0]['summary']['duration']/60,1))+" Mins. </strong>" +"</h4></b>"

# m = folium.Map(location=[6.074834613830474, 80.25749815575348],zoom_start=10, control_scale=True,tiles="cartodbpositron")
# folium.GeoJson(decoded).add_child(folium.Popup(distance_txt+duration_txt,max_width=300)).add_to(m)

In [49]:

def add_route(response, m):
    # use the response
    geometry = response['routes'][0]['geometry']
    decoded = ors.convert.decode_polyline(geometry)
    
    folium.GeoJson(decoded).add_to(m)
    
    # points = [(i[1], i[0]) for i in mls[0]]   
    
    # add marker for the start and ending points
    # for point in [points[0], points[-1]]:
    #     folium.Marker(point).add_to(m)   # add the lines
    # folium.PolyLine(points, weight=5, opacity=1).add_to(m)   # create optimal zoom
    df = pd.DataFrame(decoded['coordinates']).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
    sw = df[['Lat', 'Lon']].min().values.tolist()
    ne = df[['Lat', 'Lon']].max().values.tolist()
    m.fit_bounds([sw, ne])
    return m


In [ ]:
m = folium.Map()

In [58]:
m = add_route(result, m)

In [59]:
m